In [4]:
import tensorflow as tf
from tensorflow import keras
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

In [6]:
_labels_ = ['STANDING', 'WALKING_UPSTAIRS', 'LAYING', 'WALKING', 'SITTING', 'WALKING_DOWNSTAIRS']

In [7]:
def label_to_vector(label):
    return [int(label==l) for l in _labels_]

In [8]:
def vector_to_label(vec):
    return _labels_[vec.index(1)]

In [ ]:
def get_y(labels):
    return [label_to_vector(l) for l in labels]

In [ ]:
# build model:  -> 5 -> 6
model = keras.Sequential([
    keras.layers.Dense(5, input_dim=4232, activation="relu"),
    keras.layers.Dense(6, activation="softmax")
])

In [ ]:
# compile
optimizer = keras.optimizers.SGD(learning_rate=0.1)
model.compile(optimizer=optimizer, loss="MSE", metrics=['accuracy'])

In [ ]:
# train
model.fit(x_train, y_train, epochs=10)

In [ ]:
# evaluation
print("\n model evaluation:")
model.evaluate(x_test, y_test, verbose=1)

In [9]:
# make prediction
#predictions = model.predict(data)